# Using the debug tree

```{warning}
This debug functionality is highly experimental. Some of the information it exposes may be considered "internal" state of VW and is therefore not guaranteed to be stable between releases.
```

VW is composed of a stack of reductions that implement individual parts of the computation. Problems are solved by transforming the problem into something that has an existing reduction that can solve it. This is done by chaining reductions together. The debug tree is a tool that can be used to visualize the reduction stack and the data that is flowing through it.

## How to enable the debug tree

When the {py:class}`~vowpal_wabbit_next.Workspace` is constructed the `enable_debug_tree` argument must be set to `True`. This will cause any calls to {py:meth}`~vowpal_wabbit_next.Workspace.predict_one`, {py:meth}`~vowpal_wabbit_next.Workspace.learn_one`, or {py:meth}`~vowpal_wabbit_next.Workspace.predict_then_learn_one` to also return a {py:class}`~vowpal_wabbit_next.DebugNode` object that represents the root of the tree of computation.

The {py:class}`~vowpal_wabbit_next.DebugNode` essentially represents a snapshot of state at that place in the reduction stack. There are many properties that can be inspected, and can be seen here: {py:class}`~vowpal_wabbit_next.DebugNode`.

Let's say we just wanted to see which reductions were called for a simple default VW model. We can do this by traversing the returned tree and reading the properties of each node as we go.

In [10]:
def print_tree(node, depth=0):
    print(f"{'  '* depth} {node.name}({node.function}) pred:{node.output_prediction}")
    for child in node.children:
        print_tree(child, depth + 1)

In [15]:
import vowpal_wabbit_next as vw

workspace = vw.Workspace(enable_debug_tree=True)
parser = vw.TextFormatParser(workspace)

workspace.learn_one(parser.parse_line("0 | price:.23 sqft:.25 age:.05 2006"))
workspace.learn_one(
    parser.parse_line("1 2 'second_house | price:.18 sqft:.15 age:.35 1976")
)
workspace.learn_one(
    parser.parse_line("0 1 0.5 'third_house | price:.053 sqft:.32 age:.87 1924")
)

prediction, dbg_node = workspace.predict_one(
    parser.parse_line("| price:0.25 sqft:0.8 age:0.1")
)
print_tree(dbg_node)

 count_label(predict) pred:0.3051726520061493
   scorer-identity(predict) pred:0.3051726520061493
     gd(predict) pred:0.3051726520061493


VW uses a variety of different reductions, some of which are for collecting stats or transforming the state of the example as it flows through. In the above example `count_label` is used for reporting and therefore is not going to have a visible effect on the state of the example. 

In this example we can see that the prediction doesn't change as it propagates back up the tree. However, if we did something like change the link function used, then we could see the effect of that more clearly.



In [16]:
import vowpal_wabbit_next as vw

workspace = vw.Workspace(["--link=logistic"], enable_debug_tree=True)
parser = vw.TextFormatParser(workspace)

workspace.learn_one(parser.parse_line("0 | price:.23 sqft:.25 age:.05 2006"))
workspace.learn_one(
    parser.parse_line("1 2 'second_house | price:.18 sqft:.15 age:.35 1976")
)
workspace.learn_one(
    parser.parse_line("0 1 0.5 'third_house | price:.053 sqft:.32 age:.87 1924")
)

prediction, dbg_node = workspace.predict_one(
    parser.parse_line("| price:0.25 sqft:0.8 age:0.1")
)
print_tree(dbg_node)

 count_label(predict) pred:0.5757064819335938
   scorer-logistic(predict) pred:0.5757064819335938
     gd(predict) pred:0.3051726520061493


## Fetching the scores using the debug tree

When using `cb_explore_adf` it can be helpful to inspect the predicted scores were used to generate the exploration distribution. Using the debug tree, you can gain access to those scores.

In [1]:
from typing import Optional
import vowpal_wabbit_next as vw


workspace = vw.Workspace(["--cb_explore_adf"], enable_debug_tree=True)
parser = vw.TextFormatParser(workspace)

ex = [
    parser.parse_line("shared | s_1"),
    parser.parse_line("0:0.1:0.25 | a:0.5 b:1"),
    parser.parse_line("| a:-1 b:-0.5"),
    parser.parse_line("| a:-2 b:-1"),
]

workspace.learn_one(ex)
prediction, debug_node = workspace.predict_one(ex)


def find_cb_adf_node(node) -> Optional[vw.DebugNode]:
    if node.name == "cb_adf":
        return node

    for child in node.children:
        found = find_cb_adf_node(child)
        if found:
            return found

    return None


cb_adf_node = find_cb_adf_node(debug_node)

print(f"Action probabilities: {prediction}")
print(f"Action scores: {cb_adf_node.output_prediction}")

Action probabilities: [(2, 0.9666666388511658), (1, 0.01666666753590107), (0, 0.01666666753590107)]
Action scores: [(2, -0.07499927282333374), (1, -0.01249987818300724), (0, 0.09999903291463852)]
